# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


382


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

320


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor': 1}
""")

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

This task involves creating a table with the appropriate `PARTITION KEY`s and `CLUSTERING COLUMN`s. Here we model the data after the above query. The query requires that a table with the following columns is created: 
- sessionId
- itemInSession
- artist
- song
- length

`sessionId` forms the `PARTITION KEY` and `itemInSession` forms the `CLUSTERING KEY` because they are needed in the `WHERE` clause to help filter out the data of interest. The `PARTITION KEY` helps to distribute data across nodes while the `CLUSTERING KEY` helps to sort the data with a partition. The partition key and the clustering key together forms the `PRIMARY KEY`.


In [8]:
dd = dict(zip(pd.read_csv('event_datafile_new.csv').columns, range(11)))
dd

{'artist': 0,
 'firstName': 1,
 'gender': 2,
 'itemInSession': 3,
 'lastName': 4,
 'length': 5,
 'level': 6,
 'location': 7,
 'sessionId': 8,
 'song': 9,
 'userId': 10}

In [9]:
session.execute('DROP TABLE IF EXISTS sessions')
session.execute("""CREATE TABLE IF NOT EXISTS sessions (
sessionId int, itemInSession int, artist text, song text, length float,
PRIMARY KEY (sessionId, itemInSession))""")
                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessions (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        si, iis, art, s, l = dd['sessionId'],dd['itemInSession'], dd['artist'],dd['song'], dd['length']
        
        session.execute(query,[int(line[si]), int(line[iis]), line[art], line[s], float(line[l])] )

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
rows = session.execute("""select  artist,
                                song, 
                                length 
                                from sessions 
                                where sessionId = 338 and itemInSession = 4;""")

for row in rows:
    print(row.artist, row.song, row.length)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

This task involves creating a table with the appropriate `PARTITION KEY`s and `CLUSTERING COLUMN`s. Here we model the data after the above query. The query requires that a table with the following columns: 
- userId
- sessionId
- itemInSession
- artist
- song
- firstName
- lastName

`userId` and `sessionId` forms the `PARTITION KEY`  because they are needed in the `WHERE` clause to help filter out the data of interest. They also help distribute the data across nodes. `itemInSession` forms the `CLUSTERING KEY` because we need it to sort the songs.The partition key and the clustering key together forms the `PRIMARY KEY`.



In [12]:
session.execute('DROP TABLE IF EXISTS artist_by_user_session')

session.execute("""CREATE TABLE IF NOT EXISTS artist_by_user_session(
userId int, sessionId int ,itemInSession int, artist text, song text, firstName text, lastName text,
PRIMARY KEY ((userId, sessionId),itemInSession))""")
                

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """INSERT INTO artist_by_user_session (userId, sessionId, itemInSession, artist, song,
         firstName, lastName)"""
        query = query + " VALUES (%s, %s, %s, %s, %s,%s, %s)"
       
        us, se, art, s, iis, = dd['userId'],dd['sessionId'],dd['artist'],dd['song'],dd['itemInSession']
        f, l = dd['firstName'], dd['lastName']

        session.execute(query,[int(line[us]), int(line[se]), int(line[iis]), line[art], line[s], line[f],line[l]] )

In [14]:
rows = session.execute("""select artist, song, firstName, lastName from 
artist_by_user_session WHERE 
userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own


This task involves creating a table with the appropriate `PARTITION KEY`s and `CLUSTERING COLUMN`s. Here we model the data after the above query. The query requires that a table with the following columns is created: 
- song
- userId
- firstName
- lastName

`song` forms the `PARTITION KEY` and `userId` forms the `CLUSTERING KEY` because they are needed in the `WHERE` clause to help filter out the data of interest. The `PARTITION KEY` helps to distribute data across nodes while the `CLUSTERING KEY` helps to sort the data with a partition. The partition key and the clustering key together forms the `PRIMARY KEY`.


In [15]:
session.execute('DROP TABLE IF EXISTS song_all_hands')

session.execute("""CREATE TABLE IF NOT EXISTS song_all_hands(
song text, userId int, firstName text , lastName text, PRIMARY KEY (song, userId))
""")
                

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO song_all_hands (song, userId, firstName, lastName)"""
        query = query + " VALUES (%s, %s, %s, %s)"
        
        s, us, f, l = dd['song'],dd['userId'],dd['firstName'], dd['lastName']
        
        session.execute(query,[line[s], int(line[us]),line[f], line[l]] )

In [17]:
rows = session.execute("select firstName, lastName from song_all_hands where song = 'All Hands Against His Own'")

for row in rows:
    print(row.firstname, row.lastname)

### Drop the tables before closing out the sessions

In [18]:
tables = ['sessions', 'artist_by_user_session',  'song_all_hands']
querys = ['DROP TABLE '+table for table in tables]
for query in querys:
    session.execute(query)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()